In [222]:
import pandas as pd

In [223]:
df = pd.read_csv("result_df.csv")
#df = df.sample(120000)
df

,name,article,audited financial statements,aum,capital committed invested,cco notes,compliance other,compliance business continuity manual,conference call,conference meeting notes,...,reference notes,risk management policy,risk report,service provider information,side letter,tax information,term sheet,unaudited financial statements,valuation policy,wire instructions
0,quarterly manager blurb - 2024.xlsx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,verition commentary & risk report_september 20...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,saba capital carry neutral tail fund - risk re...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,lcm fund risk report september 2024.pdf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,abbey futures fund - monthly report_30 sep 202...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120817,high tech high capital campaign plan.doc,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120818,diane l.myer foundation investment policy.doc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120819,dlj wire letter.doc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
120820,initial donation mar 24 00.doc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [224]:
# Custom tokenizer function
def custom_tokenizer(file_name):
    # Split date-like patterns (e.g., '2018-19' -> '2018 19')
    file_name = re.sub(r'(\d{4})-(\d{2})', r'\1 \2', file_name)
    
    # Split camelCase words (e.g., 'MTHSandNVHS' -> 'MTH Sand NVHS')
    file_name = re.sub(r'([a-z])([A-Z])', r'\1 \2', file_name)
    
    # Convert file name to lower case and split by non-alphanumeric characters
    tokens = re.findall(r'\b\w+\b', file_name.lower())  # \w matches letters, numbers, and underscores
    return tokens

# Example: Splitting the data
X = df['name']  # File names
y = (df.drop(columns=['name']) > 0).astype(int)  # Multi-labels, assuming >0 means presence of a category

# Step 1: Apply TF-IDF Vectorization with custom tokenizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', tokenizer=custom_tokenizer)
X_tfidf = vectorizer.fit_transform(X)

# Step 2: Train the model
model = OneVsRestClassifier(LogisticRegression(max_iter=10000, random_state=42))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Fit the model on the training data
model.fit(X_train, y_train)

# Step 3: Predict on the test set
y_pred = model.predict(X_test)

# Step 4: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
hamming = hamming_loss(y_test, y_pred)

# Print Accuracy and Hamming Loss
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Hamming Loss: {hamming:.6f}")

# Step 5: Define keyword-based checks for additional classification
def apply_keyword_rules(file_name):
    """Keyword-based rule to detect presentation type files."""
    keywords = {
        'presentation': 'presentation',
        'report': 'report',
        'proposal': 'proposal',
        'meeting': 'meeting',
        'financial': 'financial report'
    }
    
    file_name = file_name.lower()  # Case-insensitive
    for keyword, category in keywords.items():
        if keyword in file_name:
            return category  # Return the category based on the keyword match
    return None  # If no match, return None

# Example to predict for new files
new_files = ["picton mahoney - firm financial statements 2016.pdf", "flagship historical risk data 04302022.xlsx"]
for file in new_files:
    predicted_category = apply_keyword_rules(file)
    
    if predicted_category:
        print(f"File: {file}")
        print(f"Predicted category based on keyword: {predicted_category}")
    else:
        # If no match from keyword, proceed with TF-IDF + model predictions
        X_new = vectorizer.transform([file])  # Transform the new file name
        y_pred_new = model.predict(X_new)
        
        # Extract the predicted categories based on the model output
        predicted_categories = df.columns[1:][y_pred_new[0] == 1].tolist()  # Assuming columns[1:] are categories
        
        print(f"File: {file}")
        if predicted_categories:
            print(f"Predicted categories based on model: {', '.join(predicted_categories)}")
        else:
            print("Predicted categories: None")
    print("\n")

C:\Users\gregory.fatouras\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy: 91.41%
Hamming Loss: 0.002539
File: picton mahoney - firm financial statements 2016.pdf
Predicted category based on keyword: financial report


File: flagship historical risk data 04302022.xlsx
Predicted categories based on model: risk management policy, risk report




In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import re

# Custom tokenizer function
def custom_tokenizer(file_name):
    file_name = re.sub(r'(\d{4})-(\d{2})', r'\1 \2', file_name)  # Split date-like patterns
    file_name = re.sub(r'([a-z])([A-Z])', r'\1 \2', file_name)  # Split camelCase words
    tokens = re.findall(r'\b\w+\b', file_name.lower())  # Tokenize by non-alphanumeric characters
    return tokens


X = df['name']  # File names

# Multi-label binarizer to transform labels into a binary matrix (1 if a file has a category, 0 if not)
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df.drop(columns=['name']).values)

# Step 1: Apply TF-IDF Vectorization with custom tokenizer
vectorizer = TfidfVectorizer(max_features=50000, stop_words='english', tokenizer=custom_tokenizer)
X_tfidf = vectorizer.fit_transform(X)

# Step 2: Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Step 3: Build the neural network model
model = Sequential()

# Input layer
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))  # Input layer with ReLU activation
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(256, activation='relu'))  # Hidden layer 1
model.add(Dropout(0.5))  # Dropout layer
model.add(Dense(128, activation='relu'))  # Hidden layer 2
model.add(Dropout(0.5))  # Dropout layer

# Output layer
model.add(Dense(y_train.shape[1], activation='sigmoid'))  # Sigmoid for multi-label classification

# Step 4: Set a custom learning rate and compile the model
custom_learning_rate = 0.00001  # Define your custom learning rate here
optimizer = Adam(learning_rate=custom_learning_rate)  # Pass the custom learning rate to Adam optimizer

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test))

# Step 6: Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Step 7: Predict for new files
new_files = ["picton mahoney - firm financial statements 2016.pdf", "flagship historical risk data 04302022.xlsx"]
new_files_tfidf = vectorizer.transform(new_files)

predictions = model.predict(new_files_tfidf)

# Convert predictions to categories
predicted_categories = mlb.inverse_transform(predictions > 0.5)  # Threshold predictions at 0.5 to get categories

for file, categories in zip(new_files, predicted_categories):
    print(f"File: {file}")
    if categories:
        print(f"Predicted categories: {', '.join(categories)}")
    else:
        print("Predicted categories: None")

C:\Users\gregory.fatouras\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\gregory.fatouras\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
327/378 ━━━━━━━━━━━━━━━━━━━━ 14s 289ms/step - accuracy: 0.6517 - loss: 0.6862